# Final Year Project
## Project # 3
### Group 3

### Imports

Libraries used in this notebook.

In [56]:
import afunctions as func
import matplotlib.pyplot as plt
import skimage

### Constants

Constants to access data on the directory structure of this repository.

In [57]:
IMG_PATH = './data/example_image/'
SEG_PATH = './data/example_segmentation/'

TRUTH = './data/example_ground_truth.csv'
FEATURES = '/features/features.csv'

### Functions

Functions created for this notebook.

### Exploratory Data Analysis

### Feature Extraction

#### Asymmetry

#### Border

#### Color

### Model

#### Creation

#### Data splitting

#### Feature scalling

#### Model fitting

#### Model prediction

#### Model score